In [53]:
from torch.utils.data import Dataset, DataLoader
import json
import gensim.models as g
import jieba
import re
import torch
import numpy as np  
from torch.nn.utils.rnn import pack_padded_sequence,pad_sequence

jieba.load_userdict("dict.txt")
stop_word_list = ["若有","岁月静好","或者","到底","一列","能够","zf","各种","好多","人们","最大","一句","一天","人数","值得","是因为","蔡","一个月","包括","一种","并且","千万","卧槽","少去",\
"前往","一幕","致","个别","主要","别的","一群","哪个","华晨","一边","一家","一场","团肖战","它们","至少","非要","是从","今日","吕贝克","尽量少","年月日时",
"不停","某些","不停","干嘛","唯一","一遍","人心","如今","应该","一些","您们","进群","有",'都','我','不','要', '人', '你', '好', '心', '有', '就', '和', '愿', '例', '向', '能', '还', '去', '戴', '到', '会', '们', '给', '多', '吃', '被', '为', '看', '很', '吗', '让', '对', '战', '（', '他', '等', '）', '来', '上', '想', '但', '没', '请', '做', '过', '太', '年', '泪', '从', '又', '最', '跪', '用', '着', '爱', '与', '把', '死', '再', '更', '一', '月', '得', '大', '后', '日', '并', '她', '才', '赞', '那', '逼', '跟', '号', '已', '比', '而', '快', '带', '急', '可', '中', '个', '里', '地', '买', '别', '呗', '妈', '新', '无', '小', '啥', '花', '谁', '跑', '打', '喵', '该', '哦', '下', '或', '怒', '只', '时', '挺', '事', '傻', '呀', '出', '天', '哭', '前', '完', '将', '怕', '者', '么', '听', '点', '每', '岁', '真', '祝', '您', '名', '求', '走', '它', '却', '少', '如', '住', '像', '搞', '转', '慌', '叫', '同', '高', '站', '\xa0', '起', '啦', '连', '热', '及', '病', '省', '由', '当', '万', '搜', '内', '性', '吐', 'N', '于', 'G', '香', '称', '劝', '家', '往', '拿', '先', '逆', '之', '发', '回', '牛', '东', '靠', '长', '以', '找', '正', '问', '刷', '总', '均', '因', '嗯', '封', '刚', '写', '哪', '至', '榜', '咋', '越', '嘛', '→', '早', '蔡徐坤', '钱', '唉', '棒', '呆', '卖', '倍', '喝', '各', '放', '命', '|', '杨', '元', '算', '哎', '防', '区', '忙', '干', '源', '则', '顶', '离', '允', '卫', '见', '汗', '成', '西', '市', '手', '↓', '孙', '另', '懂', '嗒', '机', '链', '贝', '难', '憨', '贺', '①', '②', '③', '瓜', '抓', '话', '路', '冲', '晕', '镜', '耶', '未', '全', '爸', '退', '开', '自', '脸', '疯', '所', '应', '待', '望', '其', '且', '服', '换', '酷', '赴', '抄', '哼', '行', '穿', '玩', '讲', '划', '宇', '经', 'X', '弱', '哈', '红', '史', '口', '治', '衰', '敢', '副', '线', '嫌', '瞎', '进', '玖', '黑', '字', '鼻', '北', '谭', '碰', '哇', '城', '挤', '头', '乱', '曾', '远', '委', '操', '臭', '四', '停', '勤', '台', '王', '抗', '尽', '门', '约', '力', '按', '共', '本', '倒', '獾', '嘴', '传', '受', '感', '④', '℃', '摘', '微', '老', '替', '某', '差', '啵', '达', '批', '⑤', 'C', '图', '位', '杀', '须', '据', '信', '既', '仍', '們', '噢', '蒸', '使', '此', '气', '留', '档', '猪', '滚', 'n', '砍', '非', '活', '囧', '药', '侠', '钟', '草', '票', '衣', '捂', '救', '错', '狗', '定', '怪', '抢', '哒', '三', '二', '处', '群', '捐', '零', '忘', '期', '即', '绝', '洗', '谈', '拉', '烊', '作', '冒', '轮', '害', '肺', '掉', '左', '强', '毒', 'V', '崽', '涨', '数', '遭', '书', '易', '车', '报', '管', '晚', '蛮', '套', '喜', '拍', '近', '仗', 'p', '锅', '第', '展', '学', '需', '语', '坐', '鸭', '女', '說', '弄', '条', '够', '马', '疼', '坏', '闹', '喷', '些', '揉', '山', '呜', '件', '佛', '累', '村', '养', '特', '’', '建', '仅', '饿', '肉', '变', '诶', '赚', '外', '赢', '辈', '捡', '霾', '科', '伴', '烦', '若', '屎', '扔', '贴', '查', '拖', '七', '嘘', '喊', '困', '咱', '收', '怼', '端', '亲', '县', '盼', '熬', '陪', '囤', '党', '含', '挂', '醋', '爆', '原', '蛇', '道', '蹲', '民', '港', '加', 'b', '丢', '度', '×', '祎', '米', '伤', '羞', '负', '接', '假', '播', '屁', '现', '次', '浪', '瞒', '镇', '办', '凭', '量', '飞', '照', '蹭', '浙', '分', '边', '存', '较', '借', '水', '］', '扛', '［', '醒', '藏', '官', '尚', '呈', '网', 'ω', '味', '盒', '生', '实', '矿', '店', '便', '厅', '补', '妳', '轻', '惹', '咬', '猫', '俩', '剧', '逛', '云', '犯', '鼠', '装', '贱', '烧', '翻', '贵', '願', '‘', '吹', '纸', '爬', '局', '广', '轶', '戳', '废', '恨', '低', '贰', '删', '控', '骗', '领', '躺', '吓', '兹', '鬼', '汤', '埃', '货', '哟', '踩', '反', '权', 'g', '何', '指', '炸', '亏', '身', '包', '蠢', '鸡', '继', '推', '岗', '相', '徐', '赌', '腰', '剩', '叭', '抽', '甜', '吖', '俺', '滴', '逃', '﹏', '⑥', '张', '重', '患', 'e', '周', '九', '福', '版', '阿', '菜', '慢', '粉', '神', '级', '任', '背', '＾', '压', '壶', '页', 'T', '吨', '缺', '酒', '间', '久', 'w', '法', '憋', '蛊', '予', '儿', '读', "'", '令', '超', '止', '谣', '五', '改', '遍', '唐', '扣', '吼', '♂', '怂', '皮', '宠', '李', '薇', '娅', '撒', '跳', '枪', '皆', '乡', '造', '脚', '籍', '撤', '谱', 'D', '邪', '主', '熟', '半', '為', '粑', '砸', '项', '卻', '潮', '排', '嗨', '旻', '抠', '呕', '附', '安', '友', '摊', '追', '饭', '化', '宁', 'x', '班', '举', '哥', '朱', '骑', '夹', '抖', '呃', '双', '挡', '迟', '陶', '撞', '物', '面', '梗', '核', '沪', '守', '组', '拟', '陈', '余', '罪', '輩', '長', '男', '散', '苦', '醉', '雾', '奔', '探', '寒', '喘', '幺', '财', '恶', '言', '满', '考', '岛', '敌', 'c', '雨', '抱', '眼', '–', '°', '脑', '型', 'A', '炖', '园', '揪', 'o', '宝', '风', '虽', '床', '淚', '占', '跨', '揍', '块', '摆', '昙', '舞', '嘎', '躲', '泡', '潭', '方', '竟', '具', '投', '己', '深', '拼', '窝', '淦', '坎', '黄', '夸', '美', '痰', '短', '两', '蛆', '除','）', '来', '上', '想', '但', '没', '请', '做', '过', '太', '年', '泪', "ω","⑤","④","有","就","和","是", "的" ,"因为", "了", "也","在","波","啊","呢","吧","nf","每到","这场","这","例","哦","嘛","吗"]
    
def read_data():
    x = []
    y = []
    y_map = {"happy":1 , "sad" :2 , "angry" : 3 , "neural" : 4, "fear" : 5 , "surprise" : 6}
    with open("virus_train.txt","r",encoding="utf-8") as f: 
        data = f.read()
    data = json.loads(data)
    for i in data:
        s = re.sub("//@.*?:","", i["content"])
        s = re.sub("(ht|f)tp(s?)\:\/\/[0-9a-zA-Z]([-.\w]*[0-9a-zA-Z])*(:(0-9)*)*(\/?)([a-zA-Z0-9\-\.\?\,\'\/\\\+&amp;%$#_]*)?","",s)
        s = re.sub("[\[\]_.!！+-=–—,$%·^『』～；\"，“”。?？＋、：V％~@#￥%……&*《》<>「」{}【】()/]","",s)
        s.replace("|","").replace("（","").replace("–","").replace("°","").replace("o","")
        if s is None:
            continue
        segment_list = jieba.cut(s,cut_all=False)
        # segment = ",".join(segment_list)
        # print(segment.split(","))
        temp_list = []
        for word in segment_list:
            if word not in stop_word_list:
                temp_list.append(word)
                
        x.append(temp_list)
        y.append(y_map[i["label"]])
        # break
    return  x , y
x , y = read_data()

print(len(x),len(y))

8606 8606


In [ ]:
def gensim_test():
    model = g.Word2Vec(x,min_count=3,vector_size=100)
    print(model)
    
    


In [54]:
word2vec = g.Word2Vec(x, 
                       vector_size=100,   # 词向量维度
                       min_count=3,      # 最小词频, 因为数据量较小, 这里卡1
                       epochs=200) 

# print(word2vec.wv.most_similar("武汉"))

In [55]:
word2vec.wv.most_similar("天使")
# word2vec.wv.most_similar("天使")
# for i ,v in word2vec.wv.key_to_index.items():
#     print(i,v  )
# word2vec.wv["的"]

[('有家', 0.5618909001350403),
 ('逆行', 0.5588632822036743),
 ('人才', 0.5316397547721863),
 ('坚守', 0.49722597002983093),
 ('勇敢', 0.4935377240180969),
 ('小米', 0.47751128673553467),
 ('国难', 0.46970078349113464),
 ('敬佩', 0.46722570061683655),
 ('感恩', 0.4635313153266907),
 ('负重', 0.46195361018180847)]

In [ ]:
a = "//@这瓜沙沙雕雕的保甜://@菜刀曦曦:武汉正在玩儿命求援，请求民间援助。新闻联播在播这个。我不是武汉人看了都有点生气。武汉人应该在家骂大街吧。 http://t.cn/A6PZCMJS"
s = re.sub("//@.*?:",""  , a)
s = re.sub("(ht|f)tp(s?)\:\/\/[0-9a-zA-Z]([-\".\w]*[0-9a-zA-Z])*(:(0-9)*)*(\/?)([a-zA-Z0-9\-\.\?\,\'\/\\\+&amp;%$#_]*)?","",s)
s = re.sub("[\[\]\|_.→!+-=——,$%^，（～；。）？、~@#￥%……&*《》<>「」{}【】()/]","",s)
print(s)

In [ ]:
a = [["哈哈"],["你是谁啊","玩儿命求援","猫", "狗"],["我是中国人"],["你是什么人","呢"]]
word2vec = g.Word2Vec(a, 
                   vector_size=64,   # 词向量维度
                   min_count=1,      # 最小词频, 因为数据量较小, 这里卡1
                   epochs=1) 
word2vec.wv.most_similar("猫")
word2vec.wv.key_to_index

In [ ]:
data_list = []
label = y
for s in x:
    vectors = []
    for w in s:
        if w in word2vec.wv.key_to_index:
            vectors.append(word2vec.wv[w])   # 将每个词替换为对应的词向量
    vectors = torch.Tensor(vectors)
    data_list.append(vectors)



In [ ]:
print(len(data_list))
print(len(y))



In [ ]:
data_list.sort(key=lambda x: len(x[0]), reverse=True) 
for i in data_list:
    print(len(i))
data_length = [len(sq[0]) for sq in data_list]


In [ ]:
class MyDataset(Dataset):
    def __init__(self):
        self.data = data_list
        self.label = y
        
    def __getitem__(self, index):
        data = self.data[index]
        label = self.label[index]
        return data, label

    def __len__(self):
        return len(self.label)

def collate_fn(data):
    """
    :param data: 第0维：data，第1维：label
    :return: 序列化的data、记录实际长度的序列、以及label列表
    """
    data.sort(key=lambda x: len(x[0]), reverse=True) # pack_padded_sequence要求要按照序列的长度倒序排列
    data_length = [len(sq[0]) for sq in data]
    x = [i[0] for i in data]
    y = [i[1] for i in data]
    data = pad_sequence(x, batch_first=True, padding_value=0)   # 用RNN处理变长序列的必要操作
    return data, torch.tensor(y, dtype=torch.float32), data_length



In [ ]:
train_data = MyDataset()
train_loader = DataLoader(train_data, batch_size=1, collate_fn=collate_fn, shuffle=True)


In [46]:
p = []
for i in word2vec.wv.key_to_index:
    print(i)

 
武汉
加油
疫情
致敬
我们
医护人员
肺炎
大家
平安
一起
自己
前线
你们
新型
确诊
新型冠状病毒
希望
口罩
健康
说
真的
病例
保护
一定
辛苦
打赢
没有
感染
这个
一线
防护
戴口罩
现在
可以
病毒
医院
一个
全国
中国
不要
做好
怎么
就是
防疫战
什么
他们
所有
医生
隔离
战斗
出门
已经
注意
知道
一直
封城
微笑
防控
患者
不是
湖北
野味
还是
平平安安
英雄
工作
不能
伟大
今天
这么
拳头
时候
政府
措施
所有人
感谢
工作者
人民
如果
这样
可能
作揖
国家
这次
看到
这种
一下
好好
相信
心心心
死亡
应该
医护
转发
传播
情况
起来
悲伤
支持
泪泪
发热
需要
地方
支援
上海
SARS
过年
但是
防御战
咱们
很多
觉得
哈哈哈
物资
家人
这些
小心
北京
严重
非典
感冒
问题
发现
注意安全
出来
还有
发布
新闻
医疗
新增
重视
出现
湖北省
病人
安徽
为什么
白条
目前
医务人员
春节
过去
超话
京东
野生动物
勤洗手
预防
对不起
话筒
不会
开始
照顾
传染
喜欢
信息
症状
冠状病毒
山东
首例
祝福
害怕
防范
费解
抱抱
春晚
挺住
控制
所以
允悲
摊手
朋友
谢谢
记得
那些
及时
那么
城市
回家
宣传
治疗
一切
别人
直接
出去
时间
武汉市
泪泪泪
这是
其他
人传人
鲜花
失望
期间
回来
一样
第一
感觉
抗击
微博
有人
看看
一级
接触
恐慌
取现
钟南山
社会
咳嗽
可怕
有效
取消
爆发
一点
真是
河南
重要
响应
家里
相关
消息
建议
人类
人员
最后
为了
个人
一急
专家
傻逼
可爱
传染病
good
广东
第一线
只有
卫健委
在家
国内
报道
白衣天使
视频
拜拜
关注
几个
安全
永远
作为
卫生
每天
只是
特别
之后
江苏
身边
成功
多少
一些
市场
生命
来自
完成
几天
恳请
开心
以及
帮助
生活
健健康康
最新
如何
非常
归来
有点
领导
组织
众志成城
骂
官方
关于
确认
疾病
他人
人群
嘻嘻
发生
疑似
病情
昨天
自我
必须
小时
同时
只能
能力
付出
各地
行为
笑
医用
之前
表示
肯定
虽然
新冠
报告
通报
外出
然后
得到
市民
身体
一次
央视
正常
终于
酒精
流感
累计
蝙蝠
救治


In [25]:
for i in x:
    print(i)


['天使']
['致敬', '心心', '小凡', '要', '做好', '防护', '措施', '哦', '致敬', '心', '大家', '出门', '记得', '戴', '口罩']
['中国', '赞', '中国', '赞', '中国', '赞']
['悲壮']
['一定', '会', '好', '起来']
['我', '看到', '有人', '说', '婆婆', '你', '两天', '发', '那么', '多', '新型冠状病毒', '消息', '我们', '很', '慌', '不要', '慌', '我', '之所以', '发', '这么', '多', '关于', '这件', '事', '信息', '就是', '在我看来', '所有', '恐慌', '都', '来自', '于', '未知', '只有', '及时', '有效', '公开', '最新', '信息', '让', '公众', '全面', '及时', '了解', '事情', '真相', '和', '进展', '才能', '做到', '不', '慌', '知道', '发生', '什么', '知道', '怎么', '避免', '发生']
['愿', '平安', '致敬', '辛苦', '啦', '愿', '平安']
['马毛', '都', '温暖', '人', '马嘉祺公益祺迹']
['正视', '恐惧', '要', '看到', '希望']
['平安', '平安', '抗击', '新型', '肺炎', '第一线', '致敬', '伟大', '逆', '行者', '们', '一定', '要', '平平安安', '心']
['武汉', '加油', '中国', '加油', '安徽', '加油']
['山火', '暴雨', '肺炎', '流感', '渐暖', '冬夜', '世界', '要', '毁灭', '吗']
['给力', '睡', '爱', '你', '哇', '很', '赞', '围观']
['偷笑', '这么', '萌', '照片', '我', '收藏']
['武汉', '名', '医务人员', '感染', '新型冠状病毒', '认真', '说', '之前', '媒体', '官方', '一直', '都', '说', '可控', '根本就是', '骗人', '已经', '要', '请', '钟南山', 